In [1]:
import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0 pyspark-shell'

# set the options to connect to our Kafka cluster
options = {
    # "kafka.sasl.jaas.config": 'org.apache.kafka.common.security.scram.ScramLoginModule required username="YnJhdmUtZmlzaC0xMTQ2MyQSvwXBuLOQsV1W7YffuC8cDaZcA3fKQwakMhnQGgg" password="MDUxNjc4YzEtYzYxNy00NTE1LWEwNWYtMDBhODRlZmE0OGJm";',
    # "kafka.sasl.mechanism": "SCRAM-SHA-256",
    # "kafka.security.protocol" : "SASL_SSL",
    "kafka.bootstrap.servers": 'localhost:9092',
    "subscribe": 'pizza-orders',
}

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("LearnSparkStreaming").master("local[2]").getOrCreate()

25/12/16 10:47:30 WARN Utils: Your hostname, DESKTOP-4VD7137 resolves to a loopback address: 127.0.1.1; using 172.26.8.82 instead (on interface eth0)
25/12/16 10:47:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/darwin/miniconda3/envs/kafka/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/darwin/.ivy2/cache
The jars for the packages stored in: /home/darwin/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e8a06818-c567-4f33-9a00-fa1563a723c4;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.0 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolution report :: resolve 1892ms :: artifacts dl 54ms
	:

25/12/16 10:47:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Calculate the average no. of pizzas (count) by shop.

In [7]:
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StringType, IntegerType, LongType, DoubleType, StructType, ArrayType, StructField

pizza_df = spark.read.format('kafka')\
    .options(**options)\
    .load()

pizza_schema = StructType([
  StructField("pizzaName", StringType()),
  StructField("additionalToppings", ArrayType(StringType())),
])

order_schema = StructType([
  StructField("address", StringType()),
  StructField("id", IntegerType()),
  StructField("name", StringType()),
  StructField("phoneNumber", StringType()),
  StructField("shop", StringType()),
  StructField("cost", DoubleType()),
  StructField("pizzas", ArrayType(pizza_schema)),
  StructField("timestamp", LongType()),
])

parsed_df = pizza_df.select("timestamp", from_json(col("value").cast("string"), order_schema).alias("value"))

In [8]:
parsed_df

DataFrame[timestamp: timestamp, value: struct<address:string,id:int,name:string,phoneNumber:string,shop:string,cost:double,pizzas:array<struct<pizzaName:string,additionalToppings:array<string>>>,timestamp:bigint>]

In [9]:
from pyspark.sql.functions import size, avg, col

# Group by shop and calculate the average size of the pizzas array
parsed_df.groupBy("value.shop").agg(avg(size(col("value.pizzas"))).alias("average_pizzas")).show(truncate=False)

+------------------------------------+-----------------+
|shop                                |average_pizzas   |
+------------------------------------+-----------------+
|Ill Make You a Pizza You Cant Refuse|4.333333333333333|
|Luigis Pizza                        |5.0              |
|Mammamia Pizza                      |4.0              |
+------------------------------------+-----------------+

